# Homework 10: Cleaning data with Regular Expressions

Time to use regular expressions!

# Hints and notes

### Opening files in subdirectories

Notice that this notebook might be **homework/**, but!!! the csvs and text files might be in **homework/scraped/** or **/homework/scraped/minutes_pdfs** or **/homework/pdfs/**. To open a file in a subdirectory, instead of having the filename be `"file.csv"` you'll just use `"some/subfolder/file.csv"`

### Opening text files

This will open up a file, read it in and show you the first 500 characters.

```python
contents = open("your-filename.txt").read()
contents[0:500]
```

> You might need `open("your-filename.txt", encoding="utf8").read()`

### Using regex

For some dumb reason you need to put `r` in front of the string you use when you're talking about regex. Just plain `"(\d\d\d)"` will usually work, but *sometimes* it won't and you'll need `r"(\d\d\d)`. It's best to just use the `r` all of the time, if you can remember!

### Using `.str.extract`

When you use `.str.extract`, you're always going to **capture one thing** and save it to a new column. You need to wrap the things you're interested in with parenthesis `(` `)`.

```python
df['phone_number'] = df['old_column'].str.extract(r"My phone number is (\d\d\d-\d\d\d-\d\d\d\d)")
```

### Setting pandas options

Pandas has a lot of options, like how many columns or rows it will show you, or how many characters it will show in a column before it stops showing you anything. Here are a few useful ones:

* `display.max_cols`: Number of columns to show at once
* `display.max_rows`: Number of rows to show at once
* `display.max_colwidth`: Maximum number of characters displayed from a string

You can set them using `pd.set_option("display.max_rows", 1000)`, for example, to show 1000 rows at a time. You can find a lot more at https://pandas.pydata.org/pandas-docs/stable/generated/pandas.set_option.html

### Regular expressions reference

I personally think http://www.regular-expressions.info/ is a wonderful wonderful reference (and tutorial), even if it's ugly! But here's a quick reference for you:

* `\d` is a digit
* `\d*` is zero or more digits 
* `\d+` is one or more digits
* `.` matches anything for ONE character
* `.*` is "give me anything forever"
* `\s` is whitespace, a.k.a. spaces and tabs
* `\w` is a word character, which includes capital and lowercase letters, numbers and hyphens.
* You can put `*` after anything, so `\w*` would mean "as many word characters as you can find"
* `\b` is a word boundary (you'll need the `r""` thing for this one)
* `( )` is a "capture group" for saving something
* `\1` is used when doing find/replace to say "put the first captured group here" (note, it's a dollar sign instead of a backslash in some editors)
* `[ABCDE]` is a character class, which means "match one of these, I don't care which"
* dollar sign means "end of the line"
* caret ^ means "beginning of the line"
* `\.` means "no really seriously I mean a period not just anything"
* You can use `\` with anything else that would normally be a special character, too, not just periods. `(` or `[` or whatever.

### Cleaning up extracted columns

Sometimes you get `\n` (newlines) or spaces or `\t` (tabs) or stuff at the beginning or the end of your column. `.str.strip()` will usually take care of that, just attach it after your `.str.extract()`

After you extract something, it's still a string even though you look at it and know it's a number. Use `.astype(int)` to turn it into an integer (no decimal) or `.astype(float)` to turn it into a float (yes decimal)

### Writing regular expressions in general

Even if I'm using regex in pandas or Python, I like to test them in my text editor with "Find." The highlighting really helps me see if I'm matching things! I also like to think "what stays the same?" when designing patterns, write those parts first, then fill in the blanks with what I want to capture.

## Importing

There might be more, I just wanted to put this up here for the `pd.set_option` part. It allows you to see a lot of content in a single column of pandas, which will be important for some parts below.

In [1]:
import re
import pandas as pd
pd.set_option('display.max_colwidth', 1500)

# Part 1: Using `.str.extract` to pull data from columns in pandas

## 1.1 H&M

Open up `hm.csv` from the `scraped` directory. I want **four new columns**:

1. `price_original`, the original price, one of the new price
2. `price_discounted`, the discounted price
3. `pct_discount`, the percent discount
4. `article_id`, the article id (from the url)

Save as **hm_cleaned.csv**.

**Note:** When you look at it, it... won't look right. I don't know why, pandas is weird. Look at the `price` column by itself using `df['price']` before you write your regex.

**Tip:** Remember that `$` is a special regex symbol! You might need to escape it.

**Tip:** When doing `.str.extract`, the whole match doesn't get captured, only what you put `()` around! Think about anchoring to different points of the string, or things in the string.

**Tip:** Not all prices have cents!

**Tip:** Your first instinct about how to compute the percent discount is probably wrong

In [2]:
df = pd.read_csv('scraped/hm.csv')
df.head()

,name,price,url
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A


In [3]:
df['price_discounted'] = df["price"].str.extract(r'\$(.*) ' ).astype(float)
df['price_discounted']

0     59.99
1      6.99
2      7.99
3      6.99
4     17.99
5      3.99
6      7.99
7      7.99
8      9.99
9     27.99
10     2.99
11     2.99
12     9.99
13    39.99
14    39.99
15     7.99
16    19.99
17     2.99
18     7.99
19    14.99
20     9.99
21     7.99
22     7.99
23     4.99
24     9.99
25     7.99
26     3.99
27     7.99
28     2.99
29     9.99
30    14.99
31     7.99
32    24.99
33    29.99
34     4.99
35    19.99
36     2.99
37    14.99
38     6.99
39     6.99
40     6.99
41    99.00
42    54.99
43     6.99
44     4.99
45     7.99
46     3.99
47     6.99
48     4.99
49     2.99
50     4.99
51     6.99
52     2.99
53    34.99
54     7.99
55    34.99
56     2.99
57    14.99
58     5.99
59    12.99
Name: price_discounted, dtype: float64

In [4]:
df['price_original'] = df['price'].str.extract(r' \$(.*)').astype(float)
df['price_original']

0     129.00
1      17.99
2      17.99
3      12.99
4      24.99
5       9.99
6      17.99
7      17.99
8      17.99
9      34.99
10      5.99
11      5.99
12     12.99
13     49.99
14     49.99
15     17.99
16     29.99
17      5.99
18     17.99
19     24.99
20     24.99
21     12.99
22     17.99
23      9.99
24     12.99
25     17.99
26      6.99
27     17.99
28      6.99
29     12.99
30     34.99
31     17.99
32     49.99
33     79.99
34     12.99
35     24.99
36      5.99
37     34.99
38     17.99
39     17.99
40     17.99
41    199.00
42     99.00
43     17.99
44     12.99
45     17.99
46      9.99
47     17.99
48     12.99
49      5.99
50      9.99
51     12.99
52      5.99
53     79.99
54     17.99
55     79.99
56      5.99
57     24.99
58     12.99
59     17.99
Name: price_original, dtype: float64

In [5]:
df['pct_discount'] = (df['price_original']-df['price_discounted']) / df['price_original'] * 100
df['pct_discount']

0     53.496124
1     61.145081
2     55.586437
3     46.189376
4     28.011204
5     60.060060
6     55.586437
7     55.586437
8     44.469150
9     20.005716
10    50.083472
11    50.083472
12    23.094688
13    20.004001
14    20.004001
15    55.586437
16    33.344448
17    50.083472
18    55.586437
19    40.016006
20    60.024010
21    38.491147
22    55.586437
23    50.050050
24    23.094688
25    55.586437
26    42.918455
27    55.586437
28    57.224607
29    23.094688
30    57.159188
31    55.586437
32    50.010002
33    62.507813
34    61.585835
35    20.008003
36    50.083472
37    57.159188
38    61.145081
39    61.145081
40    61.145081
41    50.251256
42    44.454545
43    61.145081
44    61.585835
45    55.586437
46    60.060060
47    61.145081
48    61.585835
49    50.083472
50    50.050050
51    46.189376
52    50.083472
53    56.257032
54    55.586437
55    56.257032
56    50.083472
57    40.016006
58    53.887606
59    27.793218
Name: pct_discount, dtype: float64

In [6]:
df.head()

,name,price,url,price_discounted,price_original,pct_discount
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,59.99,129.00,53.496124
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,6.99,17.99,61.145081
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,7.99,17.99,55.586437
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,6.99,12.99,46.189376
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,17.99,24.99,28.011204


In [7]:
df['article_id'] = df['url'].str.extract(r"=(\d+-[A-Z])")
df['article_id']

0     13472-N
1     35079-D
2     72462-A
3     58926-C
4     74242-A
5     70965-D
6     62818-B
7     69163-B
8     40910-C
9     69699-B
10    76916-A
11    68473-C
12    74498-A
13    41512-E
14    76727-A
15    76977-A
16    73369-A
17    59417-D
18    76797-A
19    76794-A
20    80432-A
21    78352-A
22    70658-A
23    72111-A
24    77665-A
25    70603-A
26    72262-B
27    70427-A
28    72262-A
29    77665-B
30    73380-A
31    35079-C
32    74773-A
33    75033-A
34    60637-C
35    70306-A
36    60621-B
37    69643-A
38    72455-B
39    72460-B
40    72462-B
41    71871-A
42    73734-A
43    74267-A
44    73344-A
45    73325-C
46    74264-A
47    72455-A
48    70799-B
49    68473-D
50    72111-C
51    58926-B
52    58211-A
53    75469-A
54    76982-A
55    74825-A
56    74263-A
57    78047-A
58    60637-G
59    78381-A
Name: article_id, dtype: object

In [8]:
df.head()

,name,price,url,price_discounted,price_original,pct_discount,article_id
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,59.99,129.00,53.496124,13472-N
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,6.99,17.99,61.145081,35079-D
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,7.99,17.99,55.586437,72462-A
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,6.99,12.99,46.189376,58926-C
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,17.99,24.99,28.011204,74242-A


In [9]:
 df.to_csv('hm_cleaned.csv', index=False)

## 1.2 Sci-Fi Authors

Open up `sci-fi.csv` to clean. Get rid of the `\n` on the title and and give me six new columns:

* `avg_rating`
* `rating_count`
* `total_score`
* `score_votes`
* `series` the series the book belongs to
* `series_no` the book in the series that it is

For series, I'm talking about e.g. `(The Hunger Games, #1)` is `series` "The Hunter Games" and `series_no` 1.

Save as **sci-fi_cleaned.csv**.

**Tip:** You don't need regex to clean the title - there's a special thing that removes whitespace from the beginning/end of strings

**Tip:** Remember that `(` and `)` are special characters

**BONUS:** When you make the `total_score` column, pay close attention to it. If you notice the problem, fix it.

**BONUS:** You don't need these columns to be numbers, but life would be better if they were. 

In [10]:
df = pd.read_csv('scraped/sci-fi.csv')
df.head()

,full_rating,full_score,rank,title,url
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,\nThe Handmaid's Tale\n,/book/show/38447.The_Handmaid_s_Tale
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"\nThe Hunger Games (The Hunger Games, #1)\n",/book/show/2767052-the-hunger-games
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"\nFrankenstein, or The Modern Prometheus\n",/book/show/18490.Frankenstein_or_The_Modern_Prometheus
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"\nA Wrinkle in Time (A Wrinkle in Time Quintet, #1)\n",/book/show/18131.A_Wrinkle_in_Time
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,\nThe Left Hand of Darkness\n,/book/show/18423.The_Left_Hand_of_Darkness


In [11]:
df['title'] = df['title'].str.strip()
df['title']

0                                                 The Handmaid's Tale
1                             The Hunger Games (The Hunger Games, #1)
2                              Frankenstein, or The Modern Prometheus
3                   A Wrinkle in Time (A Wrinkle in Time Quintet, #1)
4                                           The Left Hand of Darkness
5                                           Divergent (Divergent, #1)
6                                Catching Fire (The Hunger Games, #2)
7                                           The Giver (The Giver, #1)
8                                                    The Dispossessed
9                                                             Kindred
10                                     Oryx and Crake (MaddAddam, #1)
11                                  Mockingjay (The Hunger Games, #3)
12                                           The Time Traveler's Wife
13                             Doomsday Book (Oxford Time Travel, #1)
14                  

In [12]:
df['avg_rating'] = df['full_rating'].str.extract(r"(\d[.]\d+) [a-z]").astype(float)
df['avg_rating']

0     4.07
1     4.34
2     3.76
3     4.04
4     4.06
5     4.23
6     4.30
7     4.12
8     4.19
9     4.20
10    4.00
11    4.03
12    3.95
13    4.03
14    4.10
15    4.17
16    4.12
17    3.97
18    4.10
19    4.14
20    4.13
21    4.29
22    4.31
23    3.84
24    4.06
25    3.69
26    4.44
27    3.88
28    4.08
29    4.06
      ... 
70    4.41
71    4.02
72    3.83
73    3.55
74    3.78
75    4.01
76    4.46
77    4.18
78    4.05
79    4.32
80    4.48
81    3.88
82    4.19
83    4.28
84    3.99
85    4.91
86    3.86
87    4.04
88    4.01
89    3.91
90    3.74
91    4.03
92    4.05
93    3.82
94    4.19
95    4.17
96    3.78
97    4.16
98    3.85
99    3.83
Name: avg_rating, Length: 100, dtype: float64

In [13]:
df['rating_count'] = df['full_rating'].str.extract(r"— (.*) ratings")
df['rating_count'] = df['rating_count'].str.replace(",","").astype(int)
df['rating_count']

0      785502
1     5212935
2      922308
3      702272
4       77664
5     2345974
6     2049239
7     1379452
8       57605
9       53473
10     176024
11    1932930
12    1355662
13      38534
14      36316
15      46116
16      22554
17      53063
18     106206
19      31668
20      26705
21      18831
22      20045
23     793665
24     979341
25      33313
26      12743
27       1466
28      16555
29      81801
       ...   
70        336
71     206616
72        799
73       4568
74        215
75       8590
76        256
77        748
78      10873
79       3203
80        244
81       5588
82        367
83         93
84     366089
85         56
86        134
87       3160
88       4504
89       9782
90      11925
91       3338
92      57864
93        116
94       3328
95       3717
96     103428
97         97
98        111
99       3244
Name: rating_count, Length: 100, dtype: int32

In [14]:
df['total_score'] = df['full_score'].str.extract(r"[:] (\d+[,]\d+)")
df['total_score'] = df['total_score'].str.replace(",", "").astype(int)
df['total_score']

0     28539
1     27566
2     20049
3     17684
4     16070
5     12935
6     12261
7     11238
8     10246
9      9907
10     9628
11     9423
12     8250
13     8175
14     7523
15     6571
16     6418
17     6257
18     6238
19     5999
20     5933
21     5698
22     5046
23     5012
24     4899
25     4413
26     4398
27     4394
28     4249
29     4245
      ...  
70     1817
71     1775
72     1772
73     1740
74     1698
75     1692
76     1687
77     1684
78     1683
79     1575
80     1565
81     1564
82     1559
83     1495
84     1461
85     1458
86     1455
87     1420
88     1380
89     1323
90     1316
91     1301
92     1285
93     1278
94     1240
95     1197
96     1174
97     1170
98     1169
99     1155
Name: total_score, Length: 100, dtype: int32

In [15]:
df['score_votes'] = df['full_score'].str.extract(r"\n(\d+) people").astype(int)
df['score_votes']

0     292
1     282
2     205
3     185
4     165
5     134
6     128
7     117
8     107
9     104
10    102
11    100
12     88
13     88
14     81
15     69
16     68
17     70
18     68
19     64
20     64
21     61
22     54
23     53
24     54
25     48
26     48
27     45
28     47
29     46
     ... 
70     19
71     20
72     19
73     22
74     17
75     22
76     18
77     18
78     24
79     19
80     17
81     19
82     17
83     16
84     16
85     16
86     16
87     18
88     15
89     18
90     15
91     16
92     16
93     14
94     16
95     16
96     13
97     13
98     13
99     16
Name: score_votes, Length: 100, dtype: int32

In [16]:
df.head()

,full_rating,full_score,rank,title,url,avg_rating,rating_count,total_score,score_votes
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,4.07,785502,28539,292
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,4.34,5212935,27566,282
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,3.76,922308,20049,205
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,4.04,702272,17684,185
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,4.06,77664,16070,165


In [17]:
df['series'] = df['title'].str.extract(r"[a-z] [(](.*)[,]...")
df['series']

0                              NaN
1                 The Hunger Games
2                              NaN
3        A Wrinkle in Time Quintet
4                              NaN
5                        Divergent
6                 The Hunger Games
7                        The Giver
8                              NaN
9                              NaN
10                       MaddAddam
11                The Hunger Games
12                             NaN
13              Oxford Time Travel
14                             NaN
15                     The Sparrow
16                             NaN
17                  Imperial Radch
18            Dragonriders of Pern
19                       Earthseed
20              Oxford Time Travel
21                 Vorkosigan Saga
22                 Vorkosigan Saga
23                        The Host
24                       Divergent
25                             NaN
26                 Vorkosigan Saga
27                 Aurora Rhapsody
28                  

In [18]:
df['series_no'] = df['title'].str.extract(r".*[,] .(.*)[)]")
df['series_no']

0     NaN
1       1
2     NaN
3       1
4     NaN
5       1
6       2
7       1
8     NaN
9     NaN
10      1
11      3
12    NaN
13      1
14    NaN
15      1
16      1
17      1
18      1
19      1
20      2
21      2
22      7
23      1
24      2
25    NaN
26     10
27      1
28      1
29      2
     ... 
70      4
71    NaN
72      1
73    NaN
74      1
75    NaN
76      5
77      2
78     14
79    1-4
80      6
81      1
82      2
83     10
84      1
85      2
86      2
87    NaN
88    NaN
89    NaN
90      1
91    NaN
92      1
93      5
94      3
95      2
96      1
97      1
98      4
99    NaN
Name: series_no, Length: 100, dtype: object

In [19]:
df.to_csv('sci-fi.cleaned.csv', index=False)

## 1.3 Where you're just doing one of my former students' projects

Once upon a time my student Stefan did a project that involved some lawyer stuff. Most of the content was in PDFs, though! I converted them to text files and put them into the `pdfs` folder, and gave you code below to open up each of them and save their contents into a dataframe.

What a nice dataframe! I want you to add the following columns to it:

* `lawyer_app`, the applicant's lawyer (pro se means that they did it themselves, that's fine)
* `lawyer_gov`, the government's lawyer
* `judge`, the name of the judge
* `access`, whether the clearance is granted or denied (although you might miss a few)

Save as **court_cleaned.csv**.

**Note:** You can look at the original PDFs, they're also included.

**Note:** This uses a fun utility called `glob`, which is mostly fun because you use it as `glob.glob`. It's used to find files that match a certain filename pattern.

**BONUS:** You'll be happy once you get the judge, but make sure it doesn't have any extra punctuation on it.

**BONUS:** You can for some words using `.str.contains("blah")` and save it into new columns. Maybe `has_debt`, `has_bankruptcy`, etc.

> It's okay if it isn't perfect. Converting PDF into data rarely is! Usually you get 90% of it done with computers, then send people to enter the other 10% by hand.

In [40]:
import glob
filenames = glob.glob("scraped/pdfs/*.txt")
contents = [open(filename, encoding="utf8").read() for filename in filenames]
df = pd.DataFrame({'filename': filenames, 'content': contents})

In [41]:
df.head(7)

,filename,content
0,scraped/pdfs\11-02438.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-02438 \n\nFor Government: Stephanie C. Hess, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\n \n\nCOACHER, Robert E., Administrative Judge: \n\n \nApplicant has not mitigated the alcohol consumption security concerns. Eligibility \n\nfor access to classified information is denied. \n\nStatement of the Case \n\nOn June 16, 2015, the Department of Defense Consolidated Adjudications \nFacility (DOD CAF) issued Applicant a Statement of Reasons (SOR) detailing security \nconcerns under Guideline G, alcohol consumption. DOD CAF acted under Executive \nOrder (EO) 10865, Safeguarding Classified Information within Industry (February 20, \n1960), as amended; DOD Directive 5220.6, Defense Industrial Personnel Security \nClearance Review Program (January 2, 1992), as amended (Directive); and the \nadjudicative guidelines (AG) effective within the DOD on September 1, 2006. \n\n \nApplicant answered the SOR on July 14, 2015, and elected to have his case \ndecided on the written record in lieu of a hearing. Department Counsel subm..."
1,scraped/pdfs\11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n\n \n\nISCR Case No. 11-03073 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\nFor Government: Robert J. Kilmartin, Esq., Department Counsel \n\nFor Applicant: Mark S. Zaid, Esq. \n\nLOUGHRAN, Edward W., Administrative Judge: \n\n \nApplicant mitigated the financial considerations security concerns. Eligibility for \n\naccess to classified information is granted. \n \n\nStatement of the Case \n\nOn October 28, 2014, the Department of Defense (DOD) issued a Statement of \nReasons (SOR) to Applicant detailing security concerns under Guideline F, financial \nconsiderations. The action was taken under Executive Order (EO) 10865, Safeguarding \nClassified Information within Industry (February 20, 1960), as amended; DOD Directive \n5220.6, Defense Industrial Personnel Security Clearance Review Program (January 2, \n1992), as amended (Directive); and the adjudicative guidelines (AG) implemented by \nthe DOD on September 1, 2006. \n\n \nApplicant responded to the SOR on November 14, 2014, and requested a \nhearing before an administrative judge. The case was originally assigned to..."
2,scraped/pdfs\11-04909.h1.pdf.txt,"\n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n DEPARTMENT OF DEFENSE \n\n \n \n\n \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-04909 \n\n \n\nFor Government: Richard Stevens, Esq., Department Counsel \n\nFor Applicant: Pro se \n\n \n\n \nDUFFY, James F., Administrative Judge: \n\n \nApplicant mitigated \n\nconsiderations). Clearance is granted. \n\nthe security concerns under Guideline F (financial \n\nStatement of the Case \n\nOn April 5, 2015, the Department of Defense (DOD) Consolidated Adjudications \nFacility (CAF) issued Applicant a Statement of Reasons (SOR) detailing security \nconcerns under Guideline F. DOD CAF took that action under Executive Order 10865, \nSafeguarding Classified Information Within Industry, dated February 20, 1960, as \namended; DOD Directive 5220.6, Defense Industrial Personnel Security Clearance \nReview Program, dated January 2, 1992, as amended (Directive); and the adjudicative \nguidelines (AG) implemented by DOD on September 1, 2006. \n\nThe SOR detailed reasons why DOD adjudicators could not make the affirmative \nfinding under the Directive that it is clearly consistent with the national interest to g

Okay, now do the work and **make those new columns!**

In [42]:
df['lawyer_app'] = df['content'].str.extract(r"For Applicant[:\n]*(.*)[ \n]",re.I)
df['lawyer_app']

0                       Pro se 
1           Mark S. Zaid, Esq. 
2                       Pro se 
3           Mark S. Zaid, Esq. 
4                       Pro se 
5                       Pro se 
6                        Pro se
7      Ryan C. Nerney, Esquire 
8                       Pro se 
9                       Pro se 
10                      Pro Se 
11                       Pro se
12                       Pro se
13      Stephen Glassman, Esq. 
14                      Pro se 
Name: lawyer_app, dtype: object

In [43]:
df['lawyer_gov'] = df['content'].str.extract(r"For Government[:\n]*(.*)[,]", re.I)
df['lawyer_gov']

0        Stephanie C. Hess, Esq.
1      Robert J. Kilmartin, Esq.
2          Richard Stevens, Esq.
3          Julie R. Mendez, Esq.
4       Julie R. Mendez, Esquire
5           David F. Hayes, Esq.
6          James B. Norman, Esq.
7       Richard Stevens, Esquire
8             Tara Karoian, Esq.
9        Eric Borgstrom, Esquire
10        Robert Kilmartin, Esq.
11         James B. Norman, Esq.
12        Tovah Minster, Esquire
13           Erin Thompson, Esq.
14       David F. Hayes, Esquire
Name: lawyer_gov, dtype: object

In [46]:
df['judge'] = df['content'].str.extract(r"[\n]([A-Z][A-Z]*, [A-Z][a-z]*.*) Administrative Judge")
df['judge']
#df['content'].str.extract(r"[(][A-Z]{4}[)][\n]Administrative Judge ([A-Z][a-z]* [A-Z][a-z]*) denied \w+") 

0           COACHER, Robert E.,
1          LOUGHRAN, Edward W.,
2              DUFFY, James F.,
3                 HARVEY, Mark,
4     MARSHALL, Jr., Arthur E.,
5            MENDEZ, Francisco,
6                           NaN
7       GALES, Robert Robinson,
8              DUFFY, James F.,
9               HOGAN, Erin C.,
10      GOLDSTEIN, Jennifer I.,
11                          NaN
12            MOGUL, Martin H.,
13        GARCIA, Candace Le’i,
14             HOWE, Philip S.,
Name: judge, dtype: object

In [54]:
df['access'] = df['content'].str.extract(r"(is granted|is denied|declined to grant)")
df['access']

0             is denied
1            is granted
2            is granted
3            is granted
4             is denied
5             is denied
6     declined to grant
7            is granted
8             is denied
9            is granted
10            is denied
11    declined to grant
12            is denied
13           is granted
14            is denied
Name: access, dtype: object

In [55]:
df.head()

,filename,content,lawyer_app,lawyer_gov,judge,access
0,scraped/pdfs\11-02438.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-02438 \n\nFor Government: Stephanie C. Hess, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\n \n\nCOACHER, Robert E., Administrative Judge: \n\n \nApplicant has not mitigated the alcohol consumption security concerns. Eligibility \n\nfor access to classified information is denied. \n\nStatement of the Case \n\nOn June 16, 2015, the Department of Defense Consolidated Adjudications \nFacility (DOD CAF) issued Applicant a Statement of Reasons (SOR) detailing security \nconcerns under Guideline G, alcohol consumption. DOD CAF acted under Executive \nOrder (EO) 10865, Safeguarding Classified Information within Industry (February 20, \n1960), as amended; DOD Directive 5220.6, Defense Industrial Personnel Security \nClearance Review Program (January 2, 1992), as amended (Directive); and the \nadjudicative guidelines (AG) effective within the DOD on September 1, 2006. \n\n \nApplicant answered the SOR on July 14, 2015, and elected to have his case \ndecided on the written record in lieu of a hearing. Department Counsel subm...",Pro se,"Stephanie C. Hess, Esq.","COACHER, Robert E.,",is denied
1,scraped/pdfs\11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n\n \n\nISCR Case No. 11-03073 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\nFor Government: Robert J. Kilmartin, Esq., Department Counsel \n\nFor Applicant: Mark S. Zaid, Esq. \n\nLOUGHRAN, Edward W., Administrative Judge: \n\n \nApplicant mitigated the financial considerations security concerns. Eligibility for \n\naccess to classified information is granted. \n \n\nStatement of the Case \n\nOn October 28, 2014, the Department of Defense (DOD) issued a Statement of \nReasons (SOR) to Applicant detailing security concerns under Guideline F, financial \nconsiderations. The action was taken under Executive Order (EO) 10865, Safeguarding \nClassified Information within Industry (February 20, 1960), as amended; DOD Directive \n5220.6, Defense Industrial Personnel Security Clearance Review Program (January 2, \n1992), as amended (Directive); and the adjudicative guidelines (AG) implemented by \nthe DOD on September 1, 2006. \n\n \nApplicant responded to the SOR on November 14, 2014, and requested a \nhearing before an administrative judge. The case was originally assigned to...","Mark S. Zaid, Esq.","Robert J. Kilmartin, Esq.","LOUGHRAN, Edward W.,",is granted
2,scraped/pdfs\11-04909.h1.pdf.txt,"\n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n DEPARTMENT OF DEFENSE \n\n \n \n\n \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-04909 \n\n \n\nFor Government: Richard Stevens, Esq., Department Counsel \n\nFor Applicant: Pro se \n\n \n\n \nDUFFY, James F., Administrative Judge: \n\n \nApplicant mitigated \n\nconsiderations). Clearance is granted. \n\nthe security concerns under Guideline F (financial \n\nStatement of the Case \n\nOn April 5, 2015, the Department of Defense (DOD) Consolidated Adjudications \nFacility (CAF) issued Applicant a Statement of Reasons (SOR) detailing security \nconcerns under Guideline F. DOD CAF took that action under Executive Order 10865, \nSafeguarding Classified Information Within Industry, dated February 20, 1960, as \namended; DOD Directive 5220.6, Defense Industrial Personnel Security Clearance \nReview Program, dated January 2, 1992, as amended (Directive); and the adjudicative \nguidelines (AG) implemented by DOD on Septembe

# Reading books

When you're doing text work, you're legally obligated work on Jane Austen's Pride and Prejudice (at least I *think* so). Let's do some naive analysis of it!

## Read in Jane Austen's Pride and Prejudice (without moving the file!)

It's in the `data/` directory, and named `Austen_Pride.txt`.

In [56]:
contents = open("data/Austen_Pride.txt").read()
contents[0:500]

' Pride and Prejudice\nby Jane Austen\nChapter 1\nIt is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.\nHowever little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.\n"My dear Mr. Bennet," said his lady to him one day, "have you heard that Nethe'

## Look at the first 500 or so characters of it 

In [57]:
matches = re.findall(r"\b(s?he)\b", contents, re.I)
df = pd.DataFrame({'he/she':matches})
df.count()

he/she    3047
dtype: int64

In [58]:
df.head()

,he/she
0,he
1,he
2,she
3,she
4,he


## Use a regular expression to find every "he" or "she" in the book. There should be about 3000 of them.

**Tip:** Do you know about **word boundaries?** `\b` means "the beginning of end of a word."

**Tip:** You might also want to use `re.IGNORECASE`. Maybe you'll need to google it? 

**Tip:** Do NOT use `re.compile`

## Use a regular expression to find those same "he" or "she"s, but also match *the word after it*

The first four should be:

* he is
* he had
* she told
* he came

In [60]:
two_words = re.findall(r"\b(s?he\b [a-z]*)", contents, re.I)
two_words

['he is',
 'he had',
 'she told',
 'he came',
 'he agreed',
 'he is',
 'he married',
 'he may',
 'he comes',
 'she ought',
 'he comes',
 'he chooses',
 'she is',
 'She was',
 'she was',
 'she fancied',
 'He had',
 'he should',
 'she had',
 'he suddenly',
 'She has',
 'She is',
 'she times',
 'she will',
 'she will',
 'he continued',
 'he wished',
 'she began',
 'she had',
 'he spoke',
 'he left',
 'he would',
 'he eluded',
 'He was',
 'he meant',
 'He had',
 'he had',
 'he saw',
 'he wore',
 'She could',
 'he could',
 'she began',
 'he might',
 'he ought',
 'he brought',
 'he had',
 'he was',
 'he was',
 'he was',
 'he was',
 'He was',
 'he would',
 'She is',
 'he looked',
 'he withdrew',
 'She is',
 'She told',
 'she had',
 'she had',
 'he was',
 'he had',
 'He had',
 'he ',
 'he had',
 'she entered',
 'she looked',
 'he actually',
 'she was',
 'he asked',
 'he asked',
 'he did',
 'he seemed',
 'she was',
 'he inquired',
 'she was',
 'he danced',
 'he had',
 'he would',
 'he had',
 'H

## Use capture groups to save the pronoun (he/she) as one match and the word as another

The first five should look like

```
[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed')]```

In [61]:
two_words = re.findall(r"(\bs?he\b) ([a-z]*)", contents, re.I)
two_words

[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed'),
 ('he', 'is'),
 ('he', 'married'),
 ('he', 'may'),
 ('he', 'comes'),
 ('she', 'ought'),
 ('he', 'comes'),
 ('he', 'chooses'),
 ('she', 'is'),
 ('She', 'was'),
 ('she', 'was'),
 ('she', 'fancied'),
 ('He', 'had'),
 ('he', 'should'),
 ('she', 'had'),
 ('he', 'suddenly'),
 ('She', 'has'),
 ('She', 'is'),
 ('she', 'times'),
 ('she', 'will'),
 ('she', 'will'),
 ('he', 'continued'),
 ('he', 'wished'),
 ('she', 'began'),
 ('she', 'had'),
 ('he', 'spoke'),
 ('he', 'left'),
 ('he', 'would'),
 ('he', 'eluded'),
 ('He', 'was'),
 ('he', 'meant'),
 ('He', 'had'),
 ('he', 'had'),
 ('he', 'saw'),
 ('he', 'wore'),
 ('She', 'could'),
 ('he', 'could'),
 ('she', 'began'),
 ('he', 'might'),
 ('he', 'ought'),
 ('he', 'brought'),
 ('he', 'had'),
 ('he', 'was'),
 ('he', 'was'),
 ('he', 'was'),
 ('he', 'was'),
 ('He', 'was'),
 ('he', 'would'),
 ('She', 'is'),
 ('he', 'looked'),
 ('he', 'withdrew'),
 ('She', 'is'),
 ('She', 't

## Save those matches into a dataframe

You can give the column names with `columns=['pronoun', 'verb']`

In [62]:
SHe = re.findall(r"(\bs?he\b) [a-z]*", contents, re.I)
word = re.findall(r"\bs?he\b ([a-z]*)", contents, re.I)
new_df = pd.DataFrame({'pronoun':SHe, 'verb':word})
new_df.head()

,pronoun,verb
0,he,is
1,he,had
2,she,told
3,he,came
4,he,agreed


## How many times is each pronoun used?

In [63]:
new_df['pronoun'].value_counts()

she    1324
he     1056
She     325
He      234
Name: pronoun, dtype: int64

## Oh, wait, clean that up.

Make it only 'he' and 'she' lowercase.

It should be about 1600 'she' and 1300 'he'

In [64]:
new_df['pronoun'] = new_df['pronoun'].str.lower()
new_df['pronoun'].value_counts()

she    1649
he     1290
Name: pronoun, dtype: int64

## What are the top 20 most common verbs?

In [65]:
new_df['verb'].value_counts().nlargest(20)

was          372
had          371
could        172
is           139
would         94
has           70
did           67
will          50
might         46
should        41
felt          38
must          37
said          33
saw           32
added         31
thought       31
then          26
replied       22
looked        21
continued     21
Name: verb, dtype: int64

## What are the top 20 most common verbs for 'he', and the top 20 most common for 'she'

**Tip:** Don't use groupby, just filter. If you want to know how, though, you can also look at "value counts for different categories" on [this page](http://jonathansoma.com/lede/foundations-2017/classes/more-pandas/class-notes/)

In [66]:
new_df[new_df['pronoun'] == 'he'].groupby(by='verb').count().sort_values('pronoun', ascending = False).head(20)

,pronoun
verb,
had,166
was,160
is,74
has,49
could,40
would,35
did,29
should,26
must,24


In [36]:
new_df[new_df['pronoun'] == 'she'].groupby(by='verb').count().sort_values('pronoun', ascending = False).head(20)

,pronoun
verb,
was,212
had,205
could,132
is,65
would,59
did,38
felt,33
saw,29
will,26


## Who cries more, men or women? Give me a percentage answer.

**Tip:** It's `cried`, because of, you know, how books are written

In [67]:
new_df[new_df['verb'] == 'cried'].pronoun.value_counts(normalize=True)

she    0.916667
he     0.083333
Name: pronoun, dtype: float64

## How much more common is 'he' than 'she' in J.R.R. Tolkein's Fellowship of the Ring? How does that compare to Pride and Prejudice?

The book is in the same directory.